
<div style="text-align: center;">
    <h1>ReConPatch: anomaly detection</h1>
    <h3>Authors:</h3>
    <p>Dario Loi 1940849, Elena Muia 1938610, Martina Doku 1938629</p>

</div>


<div>
    <h2>0 - Introduction</h2>
    <p>This project aims to reimplement and potentially advance the ReConPatch method proposed in the paper titled  <a href="https://arxiv.org/pdf/2305.16713v3">”ReConPatch:
Anomaly Detection by Linear Modulation of Pretrained Features.”</a> This method addresses the challenge of
anomaly detection by constructing discriminative features through a linear modulation of patch features extracted
from pre-trained models and employs contrastive representation learning to collect and distribute features in a way
that produces a target-oriented and easily separable representation of the data.</p>
</div>


In [1]:
import os
from typing import Optional
from lightning import LightningDataModule, LightningModule
import torch.optim as optim
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch as nn
from tqdm import tqdm

<div>
    <h2>1.1 - Data</h2>
    <p>In this study, we used the <a href="https://arxiv.org/pdf/2305.16713v3">MVTec AD</a> dataset
and <a href="https://arxiv.org/pdf/2305.16713v3">BTAD</a> dataset for our experiments</p>
</div>


In [10]:
#data
#downloaded from https://www.kaggle.com/uciml/pima-indians-diabetes-database
#wget https://www.mvtec.com/company/research/datasets/mvtec-ad/downloads/mvtec_anomaly_detection.tar.xz


--2024-05-09 09:11:58--  https://www.mvtec.com/company/research/datasets/mvtec-ad/downloads/mvtec_anomaly_detection.tar.xz
Resolving www.mvtec.com (www.mvtec.com)... 185.243.132.235
Connecting to www.mvtec.com (www.mvtec.com)|185.243.132.235|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-05-09 09:11:58 ERROR 404: Not Found.



<div>
    <h2>1.2 - Data preprocessing</h2>
    <p>We create a unique data module to feed to the lightning module for both the datasets. The preprocessing will follow the ones specified in the sections 4.3 and 4.4 of the aforementioned paper.</p>
</div>


In [3]:

class MVTecDataModule(LightningDataModule):
    def __init__(
        self,
        data_dir: str,
        batch_size: int = 32,
        num_workers: int = 4,
        train_val_split: float = 0.8,
        shuffle: bool = True,
        pin_memory: bool = True,
        image_size: int = 256,
        normalize: Optional[transforms.Normalize] = None,
        **kwargs,
    ):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_val_split = train_val_split
        self.shuffle = shuffle
        self.pin_memory = pin_memory
        self.image_size = image_size
        self.normalize = normalize

    def setup(self, stage=None):
        # Define transformations
        transform = transforms.Compose([
            transforms.Resize((self.image_size, self.image_size)),
            transforms.ToTensor(),
            self.normalize if self.normalize else transforms.Lambda(lambda x: x),
        ])

        # Load dataset
        tot_num=0
        for subclass in ["bottle","cable","capsule","carpet","grid","hazelnut","leather","metal_nut","pill","screw","tile","toothbrush","transistor","wood","zipper"]:
            dataset = ImageFolder(os.path.join(self.data_dir, subclass,"train"), transform=transform)
            # Split dataset into train and validation sets
            num_train = int(len(dataset) * self.train_val_split)
            num_val = len(dataset) - num_train
            tot_num+=len(dataset)
            train_dataset, val_dataset = torch.utils.data.random_split(
                dataset, [num_train, num_val])
            #add the subclass to the dataset
            if subclass == "bottle":
                self.train_dataset = train_dataset
                self.val_dataset = val_dataset
            else:
                self.train_dataset = torch.utils.data.ConcatDataset([self.train_dataset,train_dataset])
                self.val_dataset = torch.utils.data.ConcatDataset([self.val_dataset,val_dataset])

        #print the number of images in the dataset
        print("number of images in train dataset",len(self.train_dataset))
        print("number of images in val dataset",len(self.val_dataset))

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=self.shuffle,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )

    def test_dataloader(self):
        # Load test dataset without shuffling
        transform = transforms.Compose([
            transforms.Resize((self.image_size, self.image_size)),
            transforms.ToTensor(),
            self.normalize if self.normalize else transforms.Lambda(lambda x: x),
        ])
        for subclass in ["bottle","cable","capsule","carpet","grid","hazelnut","leather","metal_nut","pill","screw","tile","toothbrush","transistor","wood","zipper"]:
            test_dataset = ImageFolder(os.path.join(self.data_dir, subclass,"test"), transform=transform)
            if subclass == "bottle":
                self.test_dataset = test_dataset
            else:
                self.test_dataset = torch.utils.data.ConcatDataset([self.test_dataset,test_dataset])

        print("number of images in test dataset",len(self.test_dataset))
        return DataLoader(
            test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
        )

In [4]:
data_dir=os.path.join("data","mvtec_ad")
datamod=MVTecDataModule(data_dir='data',batch_size=32,num_workers=4,train_val_split=0.8,shuffle=True,pin_memory=True,image_size=256,normalize=None)
datamod.setup()
datamod.train_dataloader()
datamod.val_dataloader()
datamod.test_dataloader()

FileNotFoundError: [Errno 2] No such file or directory: 'data/bottle/train'

NameError: name 'torch' is not defined

Iniziamo a implementare le patches da dentro il modello (da spostare probabilmente in qualche zona più ottimale)

In [ ]:
class PairwiseSimilarity(nn.Module):
    def __init__(self, sigma = 1.0):
        super(PairwiseSimilarity, self).__init__()
        self.sigma = sigma
    def forward(self,z):
        return torch.exp(-(torch.norm(z,p=2)/self.sigma))





class ReConPatch(LightningModule):
    def __init__(
        self,
    ):  
        super(ReConPatch, self).__init__() 
        self.wr_model = torch.hub.load('pytorch/vision:v0.13.0', 'wide_resnet50_2', pretrained=True)
        self.fmap = []
        def hook(module, input, output):
            self.fmap.append(output)
        self.wr_model.layer2.register_forward_hook(hook)            
        self.wr_model.layer3.register_forward_hook(hook) 
        #self.model.fc = nn.Linear(model.fc.in_features, 512)  
    def forward(self, x):
        y = self.wr_model(x)
        dimensions = (
                int(torch.Tensor([t.shape[-2] for t in self.f_map]).mean().item()),
                int(torch.Tensor([t.shape[-1] for t in self.f_map]).mean().item())
            )
        blur = nn.AvgPool2d(3, stride = 1)
        resizer = nn.AdaptiveAvgPool2d(dimensions)
        preprocess = lambda t : resizer(blur(t))
        feature_stacks = torch.cat([preprocess(m) for m in self.fmap], dim=1)
        feature_stacks_reshaped = feature_stacks.reshape(feature_stacks.shape[1], -1).T
        
        
        
    def training_step(self, batch, batch_idx):
        x, y = batch
        output_model = self(x)
        loss = nn.CrossEntropyLoss()(x,y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamP(self.parameters(), lr=1e-3)
        return optimizer

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output_model = self(x)
        loss = nn.CrossEntropyLoss()(x,y)
        self.log('train_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        output_model = self(x)
        loss = nn.CrossEntropyLoss()(x,y)
        self.log('train_loss', loss)
        return loss

Versione esterna dal modello se si vuole implementare pezzetto pezzetto senza far partire tutto il training insieme

In [ ]:
   
model = torch.hub.load('pytorch/vision:v0.10.0', 'wide_resnet50_2', pretrained=True)
f_maps = []
def hook(module, input, output)-> None:
    f_maps.append(output)
model.layer2.register_forward_hook(hook)    
model.layer3.register_forward_hook(hook) 
#model.fc = nn.Linear(model.fc.in_features, 512)
model.eval()

In [5]:
sample, target = next(iter(datamod.train_dataloader()))
with torch.no_grad():
    y = model(sample)
    

AttributeError: 'MVTecDataModule' object has no attribute 'train_dataset'

In [6]:
dimensions = (
    int(torch.Tensor([t.shape[-2] for t in f_maps]).mean().item()),
    int(torch.Tensor([t.shape[-1] for t in f_maps]).mean().item())
)

NameError: name 'f_maps' is not defined

In [7]:
blur = nn.AvgPool2d(3, stride = 1)
resizer = nn.AdaptiveAvgPool2d(dimensions)
preprocess = lambda t : resizer(blur(t))

AttributeError: module 'torch' has no attribute 'AvgPool2d'

In [8]:
feature_stacks = torch.cat([preprocess(m) for m in f_maps], dim=1)
feature_stacks_reshaped = feature_stacks.reshape(feature_stacks.shape[1], -1).T

NameError: name 'f_maps' is not defined